In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [25]:
original = pd.read_csv('airlinedelaycauses_DelayedFlights.csv')

In [26]:
original.head()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [27]:
data = original.copy()

In [28]:
data = data.rename(columns={'Unnamed: 0': 'ID'})

In [29]:
data['DayofMonth'].max()

31

In [30]:
# merge Year, Month, DayofMonth and  DayOfWeek into a single Date column
data['Date'] = pd.to_datetime(dict(
    year=data['Year'],
    month=data['Month'],
    day=data['DayofMonth']
))


In [31]:
data.drop(columns=['Year', 'Month', 'DayofMonth'], inplace=True)

In [32]:
data.rename(columns={'CRSDepTime': 'ScheduledDepTime', 'DepTime' : 'ActualDepTime', 'CRSArrTime': 'ScheduledArrTime', 'ArrTime': 'ActualArrTime', 'CRSElapsedTime' : 'ScheduledElapsedTime'}, inplace=True)

In [33]:
data['CancellationReason'] = data['CancellationCode'].map({
    'N': 'Not Cancelled',
    'A': 'Carrier',
    'B': 'Weather',
    'C': 'NAS'
})
data.drop(columns=['CancellationCode'], inplace=True)

In [34]:
data.columns

Index(['ID', 'DayOfWeek', 'ActualDepTime', 'ScheduledDepTime', 'ActualArrTime',
       'ScheduledArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum',
       'ActualElapsedTime', 'ScheduledElapsedTime', 'AirTime', 'ArrDelay',
       'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut',
       'Cancelled', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay',
       'SecurityDelay', 'LateAircraftDelay', 'Date', 'CancellationReason'],
      dtype='object')

In [35]:
data['Diverted']

0          0
1          0
2          0
3          0
4          0
          ..
1936753    0
1936754    0
1936755    0
1936756    0
1936757    0
Name: Diverted, Length: 1936758, dtype: int64

In [36]:
data['ScheduledDepTime']

0          1955
1           735
2           620
3          1755
4          1915
           ... 
1936753    1220
1936754     600
1936755     847
1936756    1240
1936757    1103
Name: ScheduledDepTime, Length: 1936758, dtype: int64

In [37]:
for col in ['ActualDepTime', 'ScheduledDepTime', 'ActualArrTime', 'ScheduledArrTime']:
    times = (
        data[col]
        .fillna(0)
        .astype(int)
        .astype(str)
        .str.zfill(4)
        .replace('2400', '0000')
    )
    data[col] = pd.to_datetime(times, format='%H%M', errors='coerce').dt.time


In [38]:
data['Date'] = pd.to_datetime(data['Date'])

In [39]:
categorical_cols = ['UniqueCarrier', 'Origin', 'Dest', 'CancellationReason', 'DayOfWeek']
data[categorical_cols] = data[categorical_cols].astype('category')

In [40]:
data['Cancelled'] = data['Cancelled'].astype(bool)
data['Diverted'] = data['Diverted'].astype(bool)

In [41]:
delay_cols = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
data[delay_cols] = data[delay_cols].fillna(0)

In [42]:
data.isnull().sum()

ID                         0
DayOfWeek                  0
ActualDepTime              0
ScheduledDepTime           0
ActualArrTime              0
ScheduledArrTime           0
UniqueCarrier              0
FlightNum                  0
TailNum                    5
ActualElapsedTime       8387
ScheduledElapsedTime     198
AirTime                 8387
ArrDelay                8387
DepDelay                   0
Origin                     0
Dest                       0
Distance                   0
TaxiIn                  7110
TaxiOut                  455
Cancelled                  0
Diverted                   0
CarrierDelay               0
WeatherDelay               0
NASDelay                   0
SecurityDelay              0
LateAircraftDelay          0
Date                       0
CancellationReason         0
dtype: int64

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1936758 entries, 0 to 1936757
Data columns (total 28 columns):
 #   Column                Dtype         
---  ------                -----         
 0   ID                    int64         
 1   DayOfWeek             category      
 2   ActualDepTime         object        
 3   ScheduledDepTime      object        
 4   ActualArrTime         object        
 5   ScheduledArrTime      object        
 6   UniqueCarrier         category      
 7   FlightNum             int64         
 8   TailNum               object        
 9   ActualElapsedTime     float64       
 10  ScheduledElapsedTime  float64       
 11  AirTime               float64       
 12  ArrDelay              float64       
 13  DepDelay              float64       
 14  Origin                category      
 15  Dest                  category      
 16  Distance              int64         
 17  TaxiIn                float64       
 18  TaxiOut               float64       
 19  

In [44]:
data.to_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv', index=False)

In [45]:
data_cleaned = pd.read_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv')

# Visualizations

In [2]:
from dash import Dash, dcc, html, Input, Output, State, ALL, ctx
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime

# load the cleaned data
data = pd.read_csv('airlinedelaycauses_DelayedFlights_Cleaned.csv')

# need to extract hour from scheduled departure time for heatmap later
# the time format might be either object (HH:MM:SS) or numeric (HHMM)
if data['ScheduledDepTime'].dtype == 'object':
    try:
        data['Hour'] = pd.to_datetime(data['ScheduledDepTime'], format='%H:%M:%S', errors='coerce').dt.hour
    except:
        # if that fails, split string and take first part
        data['Hour'] = data['ScheduledDepTime'].astype(str).str.split(':').str[0]
        data['Hour'] = pd.to_numeric(data['Hour'], errors='coerce').fillna(0).astype(int)
else:
    # numeric format like 1955 means 19:55
    data['ScheduledDepTime'] = pd.to_numeric(data['ScheduledDepTime'], errors='coerce')
    data['Hour'] = (data['ScheduledDepTime'] // 100).fillna(0).astype(int)

data['Hour'] = data['Hour'].clip(0, 23)  # make sure hour is valid
data['DayOfWeek'] = data['DayOfWeek'].astype(int)
data['Date'] = pd.to_datetime(data['Date'])

# map day numbers to names for better readability
day_names = {1: 'Mon', 2: 'Tue', 3: 'Wed', 4: 'Thu', 5: 'Fri', 6: 'Sat', 7: 'Sun'}
data['DayName'] = data['DayOfWeek'].map(day_names)

# color scheme - went with dark theme to make it easier on eyes
COLORS = {
    'bg_dark': '#0a0e27',
    'bg_medium': '#1a1f3a',
    'bg_card': '#252d4a',
    'bg_hover': '#2d3657',
    'accent_blue': '#00d4ff',
    'accent_purple': '#9d4edd',
    'accent_pink': '#ff006e',
    'accent_orange': '#fb5607',
    'accent_green': '#06ffa5',
    'accent_yellow': '#ffbe0b',
    'text_primary': '#e8e9f3',
    'text_secondary': '#a8adc7',
    'text_muted': '#6b7494',
    'border': '#3d4567',
    'success': '#06ffa5',
    'warning': '#ffbe0b',
    'error': '#ff006e',
    'grid': '#2d3657'
}

app = Dash(__name__, suppress_callback_exceptions=True)

# helper to create those KPI metric cards at top
def create_kpi_card(title, value, color, subtitle):
    return html.Div(style={
        'backgroundColor': COLORS['bg_card'],
        'padding': '20px',
        'borderRadius': '12px',
        'border': f'2px solid {color}',
        'boxShadow': f'0 4px 12px {color}33',
        'position': 'relative',
        'overflow': 'hidden'
    }, children=[
        # little gradient effect in corner
        html.Div(style={
            'position': 'absolute',
            'top': '0',
            'right': '0',
            'width': '80px',
            'height': '80px',
            'background': f'radial-gradient(circle, {color}20 0%, transparent 70%)',
            'borderRadius': '50%',
            'transform': 'translate(30%, -30%)'
        }),
        html.Div(title, style={'color': COLORS['text_secondary'], 'fontSize': '12px', 'fontWeight': '600', 'marginBottom': '8px', 'textTransform': 'uppercase', 'letterSpacing': '0.5px'}),
        html.Div(value, style={'color': color, 'fontSize': '28px', 'fontWeight': '700', 'marginBottom': '5px', 'lineHeight': '1'}),
        html.Div(subtitle, style={'color': COLORS['text_muted'], 'fontSize': '11px'})
    ])

# helper to create chart containers with info button
def create_chart_card(title, description, chart_id, height='350px'):
    return html.Div(style={
        'backgroundColor': COLORS['bg_card'], 
        'padding': '20px', 
        'borderRadius': '12px',
        'border': f'1px solid {COLORS["border"]}', 
        'boxShadow': '0 4px 12px rgba(0, 0, 0, 0.3)'
    }, children=[
        html.Div(style={'display': 'flex', 'alignItems': 'center', 'marginBottom': '15px'}, children=[
            html.H4(title, style={
                'color': COLORS['text_primary'], 
                'margin': '0', 
                'fontSize': '16px', 
                'fontWeight': '600',
                'flex': '1'
            }),
            # info button with css tooltip
            html.Div(className='tooltip-container', style={'position': 'relative'}, children=[
                html.Span('ℹ', className='info-icon', style={
                    'color': COLORS['accent_blue'],
                    'fontSize': '18px',
                    'cursor': 'help',
                    'padding': '4px 8px',
                    'borderRadius': '50%',
                    'border': f'2px solid {COLORS["accent_blue"]}',
                    'fontWeight': 'bold',
                    'display': 'inline-block'
                }),
                html.Div(description, className='tooltip-text', style={
                    'visibility': 'hidden',
                    'width': '300px',
                    'backgroundColor': COLORS['bg_medium'],
                    'color': COLORS['text_primary'],
                    'textAlign': 'left',
                    'borderRadius': '8px',
                    'padding': '12px',
                    'position': 'absolute',
                    'zIndex': '1000',
                    'right': '0',
                    'top': '35px',
                    'border': f'2px solid {COLORS["accent_blue"]}',
                    'boxShadow': '0 4px 12px rgba(0, 0, 0, 0.5)',
                    'fontSize': '13px',
                    'lineHeight': '1.5',
                    'opacity': '0',
                    'transition': 'opacity 0.3s, visibility 0.3s'
                })
            ])
        ]),
        dcc.Graph(
            id=chart_id, 
            config={'displayModeBar': True, 'displaylogo': False}, 
            style={'height': height}
        )
    ])

# main layout
app.layout = html.Div(style={
    'backgroundColor': COLORS['bg_dark'],
    'minHeight': '100vh',
    'margin': '0',
    'padding': '0',
    'fontFamily': '"Inter", -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif'
}, children=[
    # header
    html.Div(style={
        'background': f'linear-gradient(135deg, {COLORS["bg_medium"]} 0%, {COLORS["accent_purple"]} 100%)',
        'padding': '25px 40px',
        'borderBottom': f'2px solid {COLORS["accent_blue"]}',
        'boxShadow': '0 4px 20px rgba(0, 212, 255, 0.3)'
    }, children=[
        html.Div(style={'maxWidth': '1600px', 'margin': '0 auto', 'display': 'flex', 'alignItems': 'center', 'justifyContent': 'space-between', 'flexWrap': 'wrap', 'gap': '15px'}, children=[
            html.Div([
                html.H1('US Flight Delay Analysis Dashboard', 
                    style={'color': COLORS['text_primary'], 'margin': '0', 'fontSize': '32px', 'fontWeight': '700', 'letterSpacing': '-0.5px'}),
                html.P('Interactive analysis of 1.9M flights from 2008 BTS dataset', 
                    style={'color': COLORS['text_secondary'], 'margin': '8px 0 0 0', 'fontSize': '14px', 'fontWeight': '400'})
            ]),
            html.Div(style={'display': 'flex', 'gap': '10px', 'alignItems': 'center'}, children=[
                # open in browser button for vscode users
             
                html.Span('INFOB2DA PA4', style={'color': COLORS['accent_green'], 'fontSize': '12px', 'fontWeight': '600', 
                    'padding': '6px 12px', 'backgroundColor': 'rgba(6, 255, 165, 0.1)', 'borderRadius': '4px', 'border': f'1px solid {COLORS["accent_green"]}'})
            ])
        ])
    ]),
    
    # main container
    html.Div(style={'maxWidth': '1600px', 'margin': '0 auto', 'padding': '25px'}, children=[
        
        # control panel with all filters
        html.Div(style={
            'backgroundColor': COLORS['bg_card'],
            'padding': '25px',
            'borderRadius': '12px',
            'marginBottom': '25px',
            'border': f'1px solid {COLORS["border"]}',
            'boxShadow': '0 4px 12px rgba(0, 0, 0, 0.3)'
        }, children=[
            html.Div([
                html.H3('Filters & Controls', style={'color': COLORS['text_primary'], 'margin': '0 0 20px 0', 'fontSize': '18px', 'fontWeight': '600'}),
                
                # first row of filters
                html.Div(style={'display': 'grid', 'gridTemplateColumns': '1fr 2fr 1fr', 'gap': '20px', 'marginBottom': '20px'}, children=[
                    # delay type selector
                    html.Div([
                        html.Label('Delay Metric', style={'color': COLORS['text_secondary'], 'fontSize': '13px', 'fontWeight': '600', 'marginBottom': '8px', 'display': 'block'}),
                        dcc.Dropdown(
                            id='delay-type-dropdown',
                            options=[
                                {'label': 'Arrival Delay', 'value': 'ArrDelay'},
                                {'label': 'Departure Delay', 'value': 'DepDelay'}
                            ],
                            value='ArrDelay',
                            clearable=False,
                            style={'backgroundColor': COLORS['bg_medium'], 'border': f'1px solid {COLORS["border"]}', 'borderRadius': '6px'},
                            className='custom-dropdown'
                        )
                    ]),

                    # airline multi-select
                    html.Div([
                        html.Label('Airlines', style={'color': COLORS['text_secondary'], 'fontSize': '13px', 'fontWeight': '600', 'marginBottom': '8px', 'display': 'block'}),
                        dcc.Dropdown(
                            id='airline-filter',
                            options=[{'label': carrier, 'value': carrier} for carrier in sorted(data['UniqueCarrier'].unique())],
                            value=sorted(data['UniqueCarrier'].unique())[:5],  # default to first 5
                            multi=True,
                            placeholder='Select airlines...',
                            style={'backgroundColor': COLORS['bg_medium'], 'borderRadius': '6px'},
                            className='custom-dropdown'
                        )
                    ]),

                    # threshold slider
                    html.Div([
                        html.Div([
                            html.Label('Delay Threshold', style={'color': COLORS['text_secondary'], 'fontSize': '13px', 'fontWeight': '600', 'display': 'inline-block'}),
                            html.Span(id='threshold-value', style={'float': 'right', 'color': COLORS['accent_blue'], 'fontSize': '13px', 'fontWeight': '600'})
                        ], style={'marginBottom': '8px'}),
                        dcc.Slider(
                            id='delay-threshold',
                            min=0,
                            max=120,
                            step=5,
                            value=15,
                            marks={i: {'label': f'{i}m', 'style': {'color': COLORS['text_muted'], 'fontSize': '11px'}} for i in range(0, 121, 30)},
                            tooltip={'placement': 'bottom', 'always_visible': False}
                        )
                    ])
                ]),

                # date range picker (full width so its easier to use)
                html.Div(style={'marginBottom': '20px'}, children=[
                    html.Label('Date Range', style={'color': COLORS['text_secondary'], 'fontSize': '13px', 'fontWeight': '600', 'marginBottom': '10px', 'display': 'block'}),
                    html.Div(style={
                        'backgroundColor': COLORS['bg_medium'],
                        'padding': '15px',
                        'borderRadius': '8px',
                        'border': f'1px solid {COLORS["border"]}',
                        'display': 'flex',
                        'alignItems': 'center',
                        'gap': '15px',
                        'flexWrap': 'wrap'
                    }, children=[
                        html.Span('From:', style={'color': COLORS['text_primary'], 'fontSize': '13px', 'fontWeight': '500'}),
                        dcc.DatePickerSingle(
                            id='start-date',
                            date=data['Date'].min(),
                            display_format='MMM DD, YYYY',
                            style={'minWidth': '200px'}
                        ),
                        html.Span('To:', style={'color': COLORS['text_primary'], 'fontSize': '13px', 'fontWeight': '500', 'marginLeft': '10px'}),
                        dcc.DatePickerSingle(
                            id='end-date',
                            date=data['Date'].max(),
                            display_format='MMM DD, YYYY',
                            style={'minWidth': '200px'}
                        ),
                        html.Div(style={'marginLeft': 'auto', 'color': COLORS['text_muted'], 'fontSize': '12px'},
                                id='date-range-info')
                    ])
                ]),
                
                # quick action buttons
                html.Div(style={'display': 'flex', 'gap': '10px', 'marginTop': '15px', 'flexWrap': 'wrap'}, children=[
                    html.Button('Select All', id='select-all-btn', n_clicks=0,
                        style={'backgroundColor': COLORS['accent_blue'], 'color': COLORS['bg_dark'], 'border': 'none', 'padding': '10px 20px',
                               'borderRadius': '6px', 'cursor': 'pointer', 'fontSize': '13px', 'fontWeight': '600'}),
                    html.Button('Clear', id='clear-all-btn', n_clicks=0,
                        style={'backgroundColor': COLORS['bg_medium'], 'color': COLORS['text_primary'], 'border': f'1px solid {COLORS["border"]}',
                               'padding': '10px 20px', 'borderRadius': '6px', 'cursor': 'pointer', 'fontSize': '13px', 'fontWeight': '600'}),
                    html.Button('Reset', id='reset-btn', n_clicks=0,
                        style={'backgroundColor': COLORS['bg_medium'], 'color': COLORS['text_primary'], 'border': f'1px solid {COLORS["border"]}',
                               'padding': '10px 20px', 'borderRadius': '6px', 'cursor': 'pointer', 'fontSize': '13px', 'fontWeight': '600'}),
                ])
            ])
        ]),
        
        # KPI cards section
        html.Div(id='kpi-cards', style={'marginBottom': '25px'}),
        
        # these stores hold state for coordinated views (brushing & linking)
        dcc.Store(id='selected-airlines-store'),
        dcc.Store(id='selected-timerange-store'),
        dcc.Store(id='brush-selection-store'),
        
        # first row - time series and airline comparison
        html.Div(style={'display': 'grid', 'gridTemplateColumns': 'repeat(2, 1fr)', 'gap': '20px', 'marginBottom': '20px'}, children=[
            create_chart_card(
                title='Delay Trends Over Time',
                description='Time series showing mean (solid blue) and median (dotted pink) delays. Drag to select a time range to filter all other charts. Helps spot patterns and unusual spikes.',
                chart_id='timeseries-chart'
            ),
            
            create_chart_card(
                title='Airline Performance',
                description='Airlines ranked by average delay (lower is better). Click any bar to filter dashboard to that airline. Shows which carriers are most/least delayed.',
                chart_id='airline-performance-chart'
            ),
        ]),
        
        # second row - heatmap and delay causes
        html.Div(style={'display': 'grid', 'gridTemplateColumns': '1.2fr 0.8fr', 'gap': '20px', 'marginBottom': '20px'}, children=[
            create_chart_card(
                title='When Delays Happen Most',
                description='Heatmap showing average delays by day and hour. Warmer colors = more delays. Useful for finding peak delay times during the week.',
                chart_id='heatmap-chart'
            ),
            
            create_chart_card(
                title='Why Flights Get Delayed',
                description='Breakdown of delay causes (Carrier, Weather, NAS, Security, Late Aircraft). Only counts flights over the threshold. Shows what factors contribute most to delays.',
                chart_id='sunburst-chart'
            ),
        ]),
        
        # third row - multidimensional analysis
        html.Div(style={'display': 'grid', 'gridTemplateColumns': '1fr', 'gap': '20px', 'marginBottom': '20px'}, children=[
            create_chart_card(
                title='Flight Delay Patterns (Multi-Dimensional)',
                description='Each line = one flight showing Distance, Total Delay, and delay type breakdown. Color shows severity (purple = high delay). Look for parallel lines to find correlations between factors. Shows top 500 most delayed flights to reduce clutter.',
                chart_id='parallel-coords-chart',
                height='400px'
            ),
        ]),
        
        # fourth row - routes and distribution
        html.Div(style={'display': 'grid', 'gridTemplateColumns': 'repeat(2, 1fr)', 'gap': '20px'}, children=[
            create_chart_card(
                title='Worst Routes',
                description='Top 15 routes with highest average delays (min 50 flights). Format: Origin to Destination. Helps identify problem routes.',
                chart_id='routes-chart',
                height='400px'
            ),
            
            create_chart_card(
                title='Delay Consistency by Airline',
                description='Shows delay distribution for selected airlines. Box = middle 50% of delays, line = median, diamond = mean. Wider boxes = less consistent. Only shows airlines youve filtered.',
                chart_id='boxplot-chart',
                height='400px'
            ),
        ])
    ])
])

# simple callback to show threshold value
@app.callback(
    Output('threshold-value', 'children'),
    Input('delay-threshold', 'value')
)
def update_threshold_display(value):
    return f'{value} min'

# show how many days selected
@app.callback(
    Output('date-range-info', 'children'),
    [Input('start-date', 'date'),
     Input('end-date', 'date')]
)
def update_date_info(start, end):
    if start and end:
        start_dt = pd.to_datetime(start)
        end_dt = pd.to_datetime(end)
        days = (end_dt - start_dt).days
        return f'{days} days'
    return ''

# handle quick action buttons
@app.callback(
    [Output('airline-filter', 'value'),
     Output('delay-threshold', 'value'),
     Output('start-date', 'date'),
     Output('end-date', 'date')],
    [Input('select-all-btn', 'n_clicks'),
     Input('clear-all-btn', 'n_clicks'),
     Input('reset-btn', 'n_clicks')],
    [State('airline-filter', 'value')],
    prevent_initial_call=True
)
def handle_buttons(select_all, clear_all, reset, current_airlines):
    if not ctx.triggered:
        return current_airlines, 15, data['Date'].min(), data['Date'].max()
    
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]
    all_airlines = sorted(data['UniqueCarrier'].unique())
    
    if button_id == 'select-all-btn':
        return all_airlines, 15, data['Date'].min(), data['Date'].max()
    elif button_id == 'clear-all-btn':
        return [], 15, data['Date'].min(), data['Date'].max()
    else:
        return all_airlines[:5], 15, data['Date'].min(), data['Date'].max()

# main callback - updates all charts when filters change
# this is where the coordinated views magic happens
@app.callback(
    [Output('kpi-cards', 'children'),
     Output('timeseries-chart', 'figure'),
     Output('airline-performance-chart', 'figure'),
     Output('heatmap-chart', 'figure'),
     Output('sunburst-chart', 'figure'),
     Output('parallel-coords-chart', 'figure'),
     Output('routes-chart', 'figure'),
     Output('boxplot-chart', 'figure')],
    [Input('delay-type-dropdown', 'value'),
     Input('airline-filter', 'value'),
     Input('start-date', 'date'),
     Input('end-date', 'date'),
     Input('delay-threshold', 'value'),
     Input('timeseries-chart', 'selectedData'),  # for brushing
     Input('airline-performance-chart', 'clickData')]  # for linking
)
def update_dashboard(selected_delay, selected_airlines, start_date, end_date, 
                    delay_threshold, timeseries_selection, airline_click):
    
    # start with full dataset
    filtered_data = data.copy()
    
    # apply airline filter
    if selected_airlines and len(selected_airlines) > 0:
        filtered_data = filtered_data[filtered_data['UniqueCarrier'].isin(selected_airlines)]
    
    # apply date range filter
    filtered_data = filtered_data[
        (filtered_data['Date'] >= start_date) &
        (filtered_data['Date'] <= end_date)
    ]
    
    # brushing: if user dragged on time series, filter to that range
    if timeseries_selection and 'range' in timeseries_selection:
        x_range = timeseries_selection['range']['x']
        filtered_data = filtered_data[
            (filtered_data['Date'] >= x_range[0]) &
            (filtered_data['Date'] <= x_range[1])
        ]
    
    # linking: if user clicked airline bar, filter to that airline
    if airline_click:
        clicked_airline = airline_click['points'][0]['y']
        filtered_data = filtered_data[filtered_data['UniqueCarrier'] == clicked_airline]
    
    # handle empty data
    if len(filtered_data) == 0:
        empty_fig = go.Figure()
        empty_fig.update_layout(
            template='plotly_dark',
            paper_bgcolor=COLORS['bg_card'],
            plot_bgcolor=COLORS['bg_card'],
            font={'color': COLORS['text_primary']},
            annotations=[{'text': 'No data for these filters', 'xref': 'paper', 'yref': 'paper',
                         'x': 0.5, 'y': 0.5, 'showarrow': False, 'font': {'size': 20, 'color': COLORS['text_muted']}}]
        )
        return html.Div('No data'), empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig, empty_fig
    
    # calculate KPIs
    total_flights = len(filtered_data)
    delayed_flights = len(filtered_data[filtered_data[selected_delay] > delay_threshold])
    avg_delay = filtered_data[selected_delay].mean()
    median_delay = filtered_data[selected_delay].median()
    cancelled_count = filtered_data['Cancelled'].sum()
    
    kpi_cards = html.Div(style={'display': 'grid', 'gridTemplateColumns': 'repeat(5, 1fr)', 'gap': '15px'}, children=[
        create_kpi_card('Total Flights', f'{total_flights:,}', COLORS['accent_blue'], f'{(delayed_flights/total_flights*100):.1f}% delayed'),
        create_kpi_card('Avg Delay', f'{avg_delay:.1f}m', COLORS['accent_purple'], f'Median: {median_delay:.0f}m'),
        create_kpi_card('Delayed', f'{delayed_flights:,}', COLORS['accent_orange'], f'>{delay_threshold} min'),
        create_kpi_card('On-Time', f'{((total_flights-delayed_flights)/total_flights*100):.1f}%', COLORS['accent_green'], 'within threshold'),
        create_kpi_card('Cancelled', f'{cancelled_count:,}', COLORS['error'], f'{(cancelled_count/total_flights*100):.2f}%')
    ])
    
    # Chart 1: Time series
    delay_time = filtered_data.groupby('Date')[selected_delay].agg(['mean', 'median', 'count']).reset_index()
    fig_timeseries = go.Figure()
    fig_timeseries.add_trace(go.Scatter(
        x=delay_time['Date'], y=delay_time['mean'],
        mode='lines', name='Mean',
        line=dict(color=COLORS['accent_blue'], width=2),
        fill='tozeroy', fillcolor=f'rgba(0, 212, 255, 0.1)'
    ))
    fig_timeseries.add_trace(go.Scatter(
        x=delay_time['Date'], y=delay_time['median'],
        mode='lines', name='Median',
        line=dict(color=COLORS['accent_pink'], width=2, dash='dot')
    ))
    fig_timeseries.update_layout(
        template='plotly_dark',
        paper_bgcolor=COLORS['bg_card'],
        plot_bgcolor=COLORS['bg_card'],
        font={'color': COLORS['text_primary'], 'size': 11},
        margin=dict(l=40, r=20, t=10, b=40),
        hovermode='x unified',
        dragmode='select',  # enable box select for brushing
        xaxis=dict(gridcolor=COLORS['grid'], showgrid=True),
        yaxis=dict(gridcolor=COLORS['grid'], showgrid=True, title='Delay (min)'),
        legend=dict(orientation='h', yanchor='top', y=1.1, xanchor='left', x=0)
    )
    
    # Chart 2: Airline performance bars
    airline_stats = filtered_data.groupby('UniqueCarrier').agg({
        selected_delay: 'mean',
        'ID': 'count'
    }).reset_index().sort_values(selected_delay, ascending=True)
    airline_stats.columns = ['Carrier', 'AvgDelay', 'Flights']
    
    fig_airline = go.Figure()
    fig_airline.add_trace(go.Bar(
        y=airline_stats['Carrier'],
        x=airline_stats['AvgDelay'],
        orientation='h',
        marker=dict(
            color=airline_stats['AvgDelay'],
            colorscale=[[0, COLORS['accent_green']], [0.5, COLORS['accent_yellow']], [1, COLORS['accent_pink']]],
            showscale=False,
            line=dict(color=COLORS['border'], width=1)
        ),
        text=[f"{x:.1f}m" for x in airline_stats['AvgDelay']],
        textposition='outside',
        customdata=airline_stats['Flights'],
        hovertemplate='<b>%{y}</b><br>Avg: %{x:.1f}min<br>Flights: %{customdata:,}<extra></extra>'
    ))
    fig_airline.update_layout(
        template='plotly_dark',
        paper_bgcolor=COLORS['bg_card'],
        plot_bgcolor=COLORS['bg_card'],
        font={'color': COLORS['text_primary'], 'size': 11},
        margin=dict(l=50, r=70, t=10, b=40),
        xaxis=dict(gridcolor=COLORS['grid'], showgrid=True, title='Avg Delay (min)'),
        yaxis=dict(gridcolor=COLORS['grid'], showgrid=False)
    )
    
    # Chart 3: Heatmap
    heatmap_data = filtered_data.groupby(['DayOfWeek', 'Hour'])[selected_delay].mean().reset_index()
    heatmap_pivot = heatmap_data.pivot(index='DayOfWeek', columns='Hour', values=selected_delay)
    
    fig_heatmap = go.Figure(
    data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=[day_names.get(i, i) for i in heatmap_pivot.index],
        colorscale='Turbo',
        hovertemplate='%{y} @ %{x}:00<br>Avg: %{z:.1f}min<extra></extra>',
        colorbar=dict(title=dict(text='Minutes', side='right'))
    ))

    fig_heatmap.update_layout(
        template='plotly_dark',
        paper_bgcolor=COLORS['bg_card'],
        plot_bgcolor=COLORS['bg_card'],
        font={'color': COLORS['text_primary'], 'size': 11},
        margin=dict(l=50, r=20, t=10, b=40),
        xaxis=dict(title='Hour', side='bottom'),
        yaxis=dict(title='')
    )
    
    # Chart 4: Sunburst
    delay_causes_data = []
    causes = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']
    for cause in causes:
        total = filtered_data[filtered_data[selected_delay] > delay_threshold][cause].sum()
        if total > 0:
            delay_causes_data.append({'Cause': cause.replace('Delay', ''), 'Minutes': total})
    
    if delay_causes_data:
        delay_df = pd.DataFrame(delay_causes_data)
        fig_sunburst = go.Figure(go.Sunburst(
            labels=['Total'] + delay_df['Cause'].tolist(),
            parents=[''] + ['Total'] * len(delay_df),
            values=[delay_df['Minutes'].sum()] + delay_df['Minutes'].tolist(),
            marker=dict(colors=[COLORS['bg_medium'], COLORS['accent_blue'], COLORS['accent_purple'], 
                               COLORS['accent_pink'], COLORS['accent_orange'], COLORS['accent_green']]),
            hovertemplate='<b>%{label}</b><br>%{value:,.0f} min<extra></extra>'
        ))
        fig_sunburst.update_layout(
            template='plotly_dark',
            paper_bgcolor=COLORS['bg_card'],
            plot_bgcolor=COLORS['bg_card'],
            font={'color': COLORS['text_primary'], 'size': 11},
            margin=dict(l=0, r=0, t=10, b=0)
        )
    else:
        fig_sunburst = go.Figure()
        fig_sunburst.add_annotation(text='No delay data', xref='paper', yref='paper',
                                   x=0.5, y=0.5, showarrow=False, font={'size': 16, 'color': COLORS['text_muted']})
        fig_sunburst.update_layout(template='plotly_dark', paper_bgcolor=COLORS['bg_card'], plot_bgcolor=COLORS['bg_card'])
    
    # Chart 5: Parallel coordinates - only show top delayed flights to reduce clutter
    # sample most delayed flights instead of random sample
    delayed_subset = filtered_data[filtered_data[selected_delay] > 0].nlargest(500, selected_delay)
    
    if len(delayed_subset) > 0:
        dimensions = [
            dict(label='Distance', values=delayed_subset['Distance']),
            dict(label='Total Delay', values=delayed_subset[selected_delay]),
            dict(label='Carrier', values=delayed_subset['CarrierDelay']),
            dict(label='Weather', values=delayed_subset['WeatherDelay']),
            dict(label='NAS', values=delayed_subset['NASDelay']),
            dict(label='Late Aircraft', values=delayed_subset['LateAircraftDelay'])
        ]
        
        fig_parallel = go.Figure(data=go.Parcoords(
            line=dict(
                color=delayed_subset[selected_delay],
                colorscale='Plasma',
                showscale=True,
                cmin=delayed_subset[selected_delay].quantile(0.1),
                cmax=delayed_subset[selected_delay].quantile(0.9)
            ),
            dimensions=dimensions,
            labelfont=dict(color=COLORS['text_primary'], size=12),
            rangefont=dict(color=COLORS['text_secondary'], size=10)
        ))
        fig_parallel.update_layout(
            template='plotly_dark',
            paper_bgcolor=COLORS['bg_card'],
            plot_bgcolor=COLORS['bg_card'],
            font={'color': COLORS['text_primary']},
            margin=dict(l=80, r=80, t=20, b=20)
        )
    else:
        fig_parallel = go.Figure()
        fig_parallel.add_annotation(text='No delayed flights', xref='paper', yref='paper',
                                   x=0.5, y=0.5, showarrow=False, font={'size': 16, 'color': COLORS['text_muted']})
        fig_parallel.update_layout(template='plotly_dark', paper_bgcolor=COLORS['bg_card'], plot_bgcolor=COLORS['bg_card'])
    
    # Chart 6: Routes
    filtered_data['Route'] = filtered_data['Origin'] + ' to ' + filtered_data['Dest']
    route_stats = filtered_data.groupby('Route').agg({
        selected_delay: 'mean',
        'ID': 'count'
    }).reset_index()
    route_stats.columns = ['Route', 'AvgDelay', 'Flights']
    route_stats = route_stats[route_stats['Flights'] >= 50].nlargest(15, 'AvgDelay')
    
    fig_routes = go.Figure()
    fig_routes.add_trace(go.Bar(
        x=route_stats['AvgDelay'],
        y=route_stats['Route'],
        orientation='h',
        marker=dict(
            color=route_stats['AvgDelay'],
            colorscale='Reds',
            showscale=False,
            line=dict(color=COLORS['border'], width=1)
        ),
        text=[f'{x:.1f}' for x in route_stats['AvgDelay']],
        textposition='outside',
        hovertemplate='<b>%{y}</b><br>%{x:.1f} min avg<extra></extra>'
    ))
    fig_routes.update_layout(
        template='plotly_dark',
        paper_bgcolor=COLORS['bg_card'],
        plot_bgcolor=COLORS['bg_card'],
        font={'color': COLORS['text_primary'], 'size': 10},
        margin=dict(l=100, r=60, t=10, b=40),
        xaxis=dict(gridcolor=COLORS['grid'], showgrid=True, title='Avg Delay (min)'),
        yaxis=dict(gridcolor=COLORS['grid'], showgrid=False)
    )
    
    # Chart 7: Box plot - only show selected airlines to avoid clutter
    fig_box = go.Figure()
    
    # use the filtered airlines from dropdown
    if selected_airlines and len(selected_airlines) > 0:
        carriers_to_show = selected_airlines
    else:
        # if no filter, show top 10 by volume
        carriers_to_show = filtered_data['UniqueCarrier'].value_counts().head(10).index.tolist()
    
    for carrier in sorted(carriers_to_show):
        carrier_data = filtered_data[filtered_data['UniqueCarrier'] == carrier][selected_delay]
        carrier_data = carrier_data[carrier_data > 0]  # only positive delays
        if len(carrier_data) > 10:  # need enough data points
            fig_box.add_trace(go.Box(
                y=carrier_data,
                name=carrier,
                marker=dict(color=COLORS['accent_blue'], opacity=0.7),
                line=dict(color=COLORS['accent_blue']),
                boxmean='sd',  # show mean and std dev
                boxpoints='outliers'  # only show outliers
            ))
    
    fig_box.update_layout(
        template='plotly_dark',
        paper_bgcolor=COLORS['bg_card'],
        plot_bgcolor=COLORS['bg_card'],
        font={'color': COLORS['text_primary'], 'size': 11},
        margin=dict(l=40, r=20, t=10, b=60),
        yaxis=dict(gridcolor=COLORS['grid'], showgrid=True, title='Delay (min)'),
        xaxis=dict(gridcolor=COLORS['grid'], showgrid=False),
        showlegend=False
    )
    
    return kpi_cards, fig_timeseries, fig_airline, fig_heatmap, fig_sunburst, fig_parallel, fig_routes, fig_box

# custom CSS styling
app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>Flight Delay Dashboard</title>
        {%favicon%}
        {%css%}
        <style>
            @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap');
            
            /* tooltip hover effect */
            .tooltip-container:hover .tooltip-text {
                visibility: visible !important;
                opacity: 1 !important;
            }
            
            .info-icon:hover {
                background-color: ''' + COLORS['bg_hover'] + ''' !important;
            }
            
            .Select-control, .Select-menu-outer {
                background-color: ''' + COLORS['bg_medium'] + ''' !important;
                border-color: ''' + COLORS['border'] + ''' !important;
                color: ''' + COLORS['text_primary'] + ''' !important;
            }
            .Select-value-label, .Select-option {
                color: ''' + COLORS['text_primary'] + ''' !important;
            }
            .Select-option:hover {
                background-color: ''' + COLORS['bg_hover'] + ''' !important;
            }
            .DateInput, .DateInput_input, .SingleDatePickerInput {
                background-color: ''' + COLORS['bg_medium'] + ''' !important;
            }
            .DateInput_input {
                background-color: ''' + COLORS['bg_medium'] + ''' !important;
                color: ''' + COLORS['text_primary'] + ''' !important;
                border-color: ''' + COLORS['border'] + ''' !important;
                padding: 8px 12px;
                font-size: 13px;
            }
            .DateInput_input__focused {
                border-color: ''' + COLORS['accent_blue'] + ''' !important;
            }
            .CalendarDay__selected {
                background: ''' + COLORS['accent_blue'] + ''' !important;
                border-color: ''' + COLORS['accent_blue'] + ''' !important;
            }
            .CalendarDay__selected:hover {
                background: ''' + COLORS['accent_purple'] + ''' !important;
            }
            .DayPickerNavigation_button {
                border-color: ''' + COLORS['border'] + ''' !important;
            }
            .rc-slider-track {
                background-color: ''' + COLORS['accent_blue'] + ''' !important;
            }
            .rc-slider-handle {
                border-color: ''' + COLORS['accent_blue'] + ''' !important;
                background-color: ''' + COLORS['accent_blue'] + ''' !important;
            }
            .rc-slider-rail {
                background-color: ''' + COLORS['border'] + ''' !important;
            }
        </style>
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

if __name__ == '__main__':
    app.run(debug=True, jupyter_mode='inline')

In [13]:
# average DepDelay for flights from MSP to HNL
data[(data['Dest'] == 'EGE') & (data['Origin'] == 'DFW')]['DepDelay'].mean()


50.19815668202765